# Imports

In [ ]:
import numpy as np
import timeit
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# Calculate the times

In [ ]:
def matmul(size: int) -> float:
    setup_code = f"""import numpy as np
size = {size}
x = np.random.randn({size}, {size})
y = np.random.randn({size}, {size})
res = np.zeros(({size}, {size}))"""

    loop = """for i in range(size):
        for j in range(size):
            for k in range(size):
                res[i][j] += x[i][k] * y[k][j]"""
    loop_time = timeit.timeit(loop, setup=setup_code, number=1)

    numpy = """res2 = x @ y"""
    numpy_time = timeit.timeit(numpy, setup=setup_code, number=1)

    return size, loop_time, numpy_time

# Plot Time Differences

In [ ]:
reps = 10
num = 250
l_mean = np.zeros(num)
n_mean = np.zeros(num)
l_std = np.zeros(num)
n_std = np.zeros(num)
l_times = np.ndarray((reps, num))
n_times = np.ndarray((reps, num))

for i in range(reps):
    sizes = []
    # Mttelwert & Standardabweichung über 10 Iterationen bilden und plotten
    for siz in range(num):
        sz, lt, nt = matmul(size=siz)
        sizes.append(sz)
        l_times[i, siz] = lt
        n_times[i, siz] = nt

        l_mean[siz] += l_times[i, siz]
        n_mean[siz] += n_times[i, siz]

l_mean = l_mean / reps
n_mean = n_mean / reps
for entry in range(num):
    l_std[entry] = np.std(l_times[:, entry])
    n_std[entry] = np.std(n_times[:, entry])

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=sizes,
            y=l_mean,
            mode="lines+markers",
            name="Loop Times",
            error_y=dict(type="data", array=l_std, visible=True),
        ),
        go.Scatter(
            x=sizes,
            y=n_mean,
            mode="lines+markers",
            name="Numpy Times",
            error_y=dict(type="data", array=n_std, visible=True),
        ),
    ]
)

fig.update_layout(width=1500, height=1000, yaxis_type="log")

fig.show()

In [ ]:
df = pd.DataFrame(
    {
        "size": sizes,
        "loop": l_mean,
        "numpy": n_mean,
    }
)

df = df.melt(id_vars="size", value_vars=["loop", "numpy"])

df["e"] = np.append(l_std, n_std)


px.scatter(df, x="size", y="value", color="variable", log_y=True, error_y="e")

# Ergebnis df als json abspeichern (plot?)

In [ ]:
df